https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/examples/notebooks/custom_layers.ipynb



In [2]:
import tensorflow as tf

tf.enable_eager_execution()

## Layers: common sets of useful operations
Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

In the tf.keras.layers package, layers are objects. To construct a layer, simply construct the object. 

Most layers take as a first argument the number of output dimensions / channels.

In [6]:
layer = tf.keras.layers.Dense(100)  # 100指定了输出维度
type(layer)

tensorflow.python.keras.layers.core.Dense

The number of input dimensions is often unnecessary, as it can be inferred the first time the layer is used, but it can be provided if you want to specify it manually, which is useful in some complex models.

In [7]:
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

In [8]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: id=50, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [9]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_3/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.6242633 ,  0.36147523, -0.1431292 , -0.32359353,  0.01444876,
          0.28876984, -0.12720239,  0.55321413,  0.3142482 , -0.0489282 ],
        [-0.13301593, -0.0158633 ,  0.04746962,  0.0040918 , -0.40805823,
          0.08727014, -0.4273607 , -0.47246763, -0.51467365, -0.4506082 ],
        [-0.57103974,  0.18835014,  0.2003457 , -0.17082858, -0.28520125,
          0.20919341,  0.34712416, -0.25119993,  0.10938066, -0.43051887],
        [ 0.4026925 ,  0.02685809, -0.3404439 ,  0.23248672,  0.4420274 ,
         -0.331929  , -0.5981807 ,  0.5543001 ,  0.10197508,  0.00768691],
        [-0.18599209,  0.07625949,  0.12279665,  0.14112836, -0.5820571 ,
         -0.28055677,  0.33070445,  0.13188833, -0.004888  , -0.52633286]],
       dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [10]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_3/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.6242633 ,  0.36147523, -0.1431292 , -0.32359353,  0.01444876,
          0.28876984, -0.12720239,  0.55321413,  0.3142482 , -0.0489282 ],
        [-0.13301593, -0.0158633 ,  0.04746962,  0.0040918 , -0.40805823,
          0.08727014, -0.4273607 , -0.47246763, -0.51467365, -0.4506082 ],
        [-0.57103974,  0.18835014,  0.2003457 , -0.17082858, -0.28520125,
          0.20919341,  0.34712416, -0.25119993,  0.10938066, -0.43051887],
        [ 0.4026925 ,  0.02685809, -0.3404439 ,  0.23248672,  0.4420274 ,
         -0.331929  , -0.5981807 ,  0.5543001 ,  0.10197508,  0.00768691],
        [-0.18599209,  0.07625949,  0.12279665,  0.14112836, -0.5820571 ,
         -0.28055677,  0.33070445,  0.13188833, -0.004888  , -0.52633286]],
       dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

## Implementing custom layers
TensorFlow 提供了便利的基类 tf.keras.layers.Layer。您可以通过继承它实现自己的层：

The best way to implement your own layer is extending the `tf.keras.layers.Layer` class and implementing:

* `__init__` , where you can do all input-independent initialization
* `build`, where you know the shapes of the input tensors and can do the rest of the initialization
* `call`, where you do the forward computation

In [11]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_variable("kernel",
                                    shape=[int(input_shape[-1]),
                                           self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)
print(layer(tf.zeros([10, 5])))
print(layer.trainable_variables)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 0.5373625 ,  0.4484678 ,  0.5988926 ,  0.27661514,  0.49361652,
         0.562203  ,  0.25794202, -0.6036055 ,  0.5633847 ,  0.48801762],
       [-0.16616648, -0.37794575, -0.5100892 , -0.5995496 ,  0.5747586 ,
        -0.22950682,  0.44296044, -0.6005403 ,  0.21082252, -0.52177525],
       [ 0.05716884, -0.32144508, -0.6048095 ,  0.0656023 , -0.5737327 ,
        -0.5962337 ,  0.10014254,  0.45625693, -0.26341763,  0.24943012],
       [-0.15098336, -0.42382127, -0.07473701,  0.32558107, -0.06657565,
        -0.5622134 , -0.33533457, 

## Models: composing layers
Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut.

The main class used when creating a layer-like thing which contains other layers is tf.keras.Model. Implementing one is done by inheriting from tf.keras.Model.